In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [90.7 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Ge

In [ ]:
#Start Spark session
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("DataFrameBasics").getOrCreate()


In [ ]:
#Read data from S3 Buckets
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-online/module_16/food.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("food.csv"), sep=",", header=True)

In [ ]:
#df.show()
#df.describe()
df.printSchema()

root
 |-- food: string (nullable = true)
 |-- price: string (nullable = true)



In [ ]:
#Import struct fields that we can use
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

We'll create the schema by creating a `StructType`, which is one of Spark's complex types, like an array or map. The `StructField` will define the column name, the data type held, and a Boolean to define whether null values will be included or not:

In [ ]:
#Create a list of struct fields
schema = [StructField("food", StringType(), True), StructField("price", IntegerType(), True),]
schema

[StructField('food', StringType(), True),
 StructField('price', IntegerType(), True)]

the code will pass the schema just created as fields in a `StructType`. All this will be stored in a variable called `final`

In [ ]:
#Pass in our fields
final = StructType(fields=schema)
final

StructType([StructField('food', StringType(), True), StructField('price', IntegerType(), True)])

Now that we have a predefined schema, we can read in the data again, only this time passing in our own schema

In [ ]:
#Read data with new schema
dataframe = spark.read.csv(SparkFiles.get("food.csv"), schema=final, sep=",", header=True)
dataframe.printSchema()

root
 |-- food: string (nullable = true)
 |-- price: integer (nullable = true)



In [ ]:
dataframe.select("price").show()


+-----+
|price|
+-----+
|    0|
|   12|
|   10|
+-----+



In [ ]:
#add new column
dataframe.withColumn('newprice', dataframe['price']).show()

#update column name
dataframe.withColumnRenamed('price', 'newerprice').show()

#double price
dataframe.withColumn('doubleprice', dataframe['price'] * 2).show()

#add dollor to the price
dataframe.withColumn('pricePlusOne', dataframe["price"] + 1).show()

#half the price
dataframe.withColumn('halfprice', dataframe["price"] / 2).show()

+-------+-----+--------+
|   food|price|newprice|
+-------+-----+--------+
|  pizza|    0|       0|
|  sushi|   12|      12|
|chinese|   10|      10|
+-------+-----+--------+

+-------+----------+
|   food|newerprice|
+-------+----------+
|  pizza|         0|
|  sushi|        12|
|chinese|        10|
+-------+----------+

+-------+-----+-----------+
|   food|price|doubleprice|
+-------+-----+-----------+
|  pizza|    0|          0|
|  sushi|   12|         24|
|chinese|   10|         20|
+-------+-----+-----------+

+-------+-----+------------+
|   food|price|pricePlusOne|
+-------+-----+------------+
|  pizza|    0|           1|
|  sushi|   12|          13|
|chinese|   10|          11|
+-------+-----+------------+

+-------+-----+---------+
|   food|price|halfprice|
+-------+-----+---------+
|  pizza|    0|      0.0|
|  sushi|   12|      6.0|
|chinese|   10|      5.0|
+-------+-----+---------+



now we'll start a new spark session

In [ ]:
spark = SparkSession.builder.appName("DataFrameFunctions").getOrCreate()

#Read data from S3 buckets
url ="https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-online/module_16/wine.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("wine.csv"), sep=",", header=True)

#show DF
df.show()

+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|country|         description|         designation|points|price|          province|            region_1|         region_2|           variety|              winery|
+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|     US|This tremendous 1...|   Martha's Vineyard|    96|  235|        California|         Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|
|  Spain|Ripe aromas of fi...|Carodorum Selecci...|    96|  110|    Northern Spain|                Toro|             null|     Tinta de Toro|Bodega Carmen Rod...|
|     US|Mac Watson honors...|Special Selected ...|    96|   90|        California|      Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley|
|     US|This spent 20

In [ ]:
#to select a column/row element its using the select method
#example: df.select("country").collect()[0]

In [ ]:
#order df by ascending values
df.orderBy(df["points"].desc()).show(5)

+-------+--------------------+--------------------+------+-----+----------+-----------+--------+--------------------+--------------------+
|country|         description|         designation|points|price|  province|   region_1|region_2|             variety|              winery|
+-------+--------------------+--------------------+------+-----+----------+-----------+--------+--------------------+--------------------+
|     US|This is an absolu...|           IX Estate|    99|  290|California|Napa Valley|    Napa|           Red Blend|              Colgin|
| France|98-100 Barrel sam...|       Barrel sample|    99| null|  Bordeaux|   Pauillac|    null|Bordeaux-style Re...|Ch̢teau Pontet-Canet|
|     US|There are incredi...|Elevation 1147 Es...|    99|  150|California|Napa Valley|    Napa|  Cabernet Sauvignon|        David Arthur|
| France|A magnificent Cha...|Dom P̩rignon Oeno...|    99|  385| Champagne|  Champagne|    null|     Champagne Blend|     Mo��t & Chandon|
|  Italy|Even better than .

In [ ]:
from pyspark.sql.functions import avg
df.select(avg("points")).show()


+-----------------+
|      avg(points)|
+-----------------+
|87.88834105383143|
+-----------------+



In [ ]:
#Filter
df.filter("price<20").show(5)

#filter by price on select columns
df.filter("price<20").select(['points', 'country', 'winery', 'price']).show(5)

#filter on exact state and price
df.filter("country = 'US' and price < 20").show()

#filter for price > 15 from california
df.filter("country = 'US' and province = 'California' and price > 15").show()

+--------+--------------------+-----------+------+-----+----------+--------------------+-----------------+--------------+--------------------+
| country|         description|designation|points|price|  province|            region_1|         region_2|       variety|              winery|
+--------+--------------------+-----------+------+-----+----------+--------------------+-----------------+--------------+--------------------+
|Bulgaria|This Bulgarian Ma...|    Bergul̩|    90|   15|  Bulgaria|                null|             null|        Mavrud|        Villa Melnik|
|   Spain|Earthy plum and c...|     Amandi|    90|   17|   Galicia|       Ribeira Sacra|             null|       Menc�_a|      Don Bernardino|
|      US|There's a lot to ...|       null|    90|   18|California|Russian River Valley|           Sonoma|    Chardonnay|            De Loach|
|      US|Massively fruity,...|       null|    91|   19|    Oregon|   Willamette Valley|Willamette Valley|    Pinot Gris|   Trinity Vineyards|